<a href="https://colab.research.google.com/github/Yanina-Kutovaya/RecSys-retail/blob/main/notebooks/service_test_main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#!curl "https://awscli.amazonaws.com/awscli-exe-linux-x86_64.zip" -o /tmp/awscliv2.zip
#!unzip -q /tmp/awscliv2.zip -d /tmp
#!rm /tmp/awscliv2.zip
#!sudo /tmp/aws/install --update
#!rm -rf /tmp/aws/ 
#!aws configure
#region eu-west-1

In [2]:
#!git clone https://github.com/Yanina-Kutovaya/RecSys-retail.git
#!pip install -r RecSys-retail/requirements_Colab.txt

In [3]:
%cd RecSys-retail

/content/RecSys-retail


In [4]:
#!dvc pull

In [5]:
import sys
import os

sys.path.append(os.getcwd())
sys.path.append(os.path.join(os.getcwd(), "service"))

import logging
import pandas as pd
from fastapi.testclient import TestClient

from main import app, Model
from src.recsys_retail.models.serialize import load
from src.recsys_retail.data.make_dataset import load_data
from src.recsys_retail.models.train import data_preprocessing_pipeline
from scripts.train_save_model import train_store

In [6]:
import warnings
warnings.filterwarnings("ignore")

In [7]:
logger = logging.getLogger(__name__)

client = TestClient(app)

MODEL = os.getenv("MODEL", default="baseline_v1")

In [8]:
def test_healthcheck():
    response = client.get("/")
    assert response.status_code == 200
    assert response.json()["status"] == "Green"
    logger.info(f'status_code = 200, status = "Green"')

In [9]:
def test_predict():
    data, item_features, user_features = load_data()
    train_dataset_lvl_2 = data_preprocessing_pipeline(
        data, item_features, user_features
    )
    train_store(train_dataset_lvl_2, "baseline_v1")
    Model.classifier = load(MODEL)

    user = {"user_id": 1340}
    response = client.post("/predict?user_id=1340", json=user)
    assert response.status_code == 200
    assert response.json()[0]["user_id"] == 1340

    logger.info(
        f"test_predict single user status_code = 200, response: {response.json()}"
    )

    users = {"user_ids": [1340, 1364]}
    response = client.post("/predict_user_list?batch_id=1", json=users)
    assert response.status_code == 200
    assert response.json()[0]["user_ids"] == [1340, 1364]

    logger.info(
        f"test_predict user_list status_code = 200, response: {response.json()}"
    )

In [10]:
test_healthcheck()
test_predict()

  0%|          | 0/15 [00:00<?, ?it/s]

  0%|          | 0/2501 [00:00<?, ?it/s]

[LightGBM] [Warning] Find whitespaces in feature_names, replace with underlines
[LightGBM] [Info] Number of positive: 6573, number of negative: 164677
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.353915 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 35458
[LightGBM] [Info] Number of data points in the train set: 171250, number of used features: 259
[LightGBM] [Warning] Find whitespaces in feature_names, replace with underlines
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.038382 -> initscore=-3.221016
[LightGBM] [Info] Start training from score -3.221016
Training until validation scores don't improve for 30 rounds
Early stopping, best iteration is:
[461]	training's auc: 0.950998	valid_1's auc: 0.939542
